# 탈옥

[baekjoon의 '탈옥' 링크](https://www.acmicpc.net/problem/9376)

### 문제   
상근이는 감옥에서 죄수 두 명을 탈옥시켜야 한다. 이 감옥은 1층짜리 건물이고, 상근이는 방금 평면도를 얻었다.

평면도에는 모든 벽과 문이 나타나있고, 탈옥시켜야 하는 죄수의 위치도 나타나 있다. 감옥은 무인 감옥으로 죄수 두 명이 감옥에 있는 유일한 사람이다.

문은 중앙 제어실에서만 열 수 있다. 상근이는 특별한 기술을 이용해 제어실을 통하지 않고 문을 열려고 한다. 하지만, 문을 열려면 시간이 매우 많이 걸린다. 두 죄수를 탈옥시키기 위해서 열어야 하는 문의 개수를 구하는 프로그램을 작성하시오.

### 입력
첫째 줄에 테스트 케이스의 개수가 주어진다. 테스트 케이스의 수는 100개를 넘지 않는다.

첫째 줄에는 평면도의 높이 h와 너비 w가 주어진다. (2 ≤ h, w ≤ 100) 다음 h개 줄에는 감옥의 평면도 정보가 주어지며, 빈 공간은 '.', 지나갈 수 없는 벽은 '\*', 문은 '#', 죄수의 위치는 '$'이다.

상근이는 감옥 밖을 자유롭게 이동할 수 있고, 평면도에 표시된 죄수의 수는 항상 두 명이다. 각 죄수와 감옥의 바깥을 연결하는 경로가 항상 존재하는 경우만 입력으로 주어진다.

### 출력
각 테스트 케이스마다 두 죄수를 탈옥시키기 위해서 열어야 하는 문의 최솟값을 출력한다.  

In [1]:
from queue import PriorityQueue

def simulation(h, w, prison, prisoner): # 시뮬레이션
    q = PriorityQueue() # 우선순위 큐
    visited = [[[h*w for _ in range(w)] for _ in range(h)] for _ in range(2)] # 해당 위치에 도달하기 위한 최소 움직임 수
    direction = [(0,1), (0,-1), (1,0), (-1,0)] # 이동방향
    answer = h*w # 답
    for i in range(2):
        y, x = prisoner[i] # i번 죄수의 좌표
        visited[i][y][x] = 0 # i번 죄수는 (y,x)의 위치에 0번째 이동에 도달함
        q.put([0, 0, y, x, i, set()]) # [통과한 문 개수, 이동 횟수, 현재 y좌표, 현재 x좌표, 죄수번호, 통과한 문 좌표] 정보 큐에 push
    opened = [[] for _ in range(2)] # 탈옥 시 각 죄수별로 지나야하는 문의 경우의 수
    while not q.empty():
        door_num, move_num, y, x, idx, doors = q.get() # [통과한 문 개수, 이동 횟수, 현재 y좌표, 현재 x좌표, 죄수번호, 통과한 문 좌표] 정보 큐에서 pop
        if (y == 0 or y == h-1 or x == 0 or x == w-1): # 탈옥 성공했을 때
            opened[idx].append(doors) # 각 죄수 별로 탈옥 성공 시 통과한 문을 저장해 놓는다.
            continue
        for add_y, add_x in direction: # 이동방향에 대해서
            if 0 <= add_y+y < h and 0 <= add_x+x < w and visited[idx][add_y+y][add_x+x] > move_num+1 and prison[add_y+y][add_x+x] != '*':
            # 벽과 이미 이동한 위치를 제외한 지도 내의 범위에서
                visited[idx][add_y+y][add_x+x] = move_num+1 # 이동했음을 표시
                if prison[add_y+y][add_x+x] == '#': # 만약 이동할 공간이 문이라면
                    q.put([door_num+1, move_num+1, add_y+y, add_x+x, idx, doors.union(set([(add_y+y, add_x+x)]))])
                    # [통과한 문 개수+1, 이동 횟수+1, 이동할 y좌표, 이동할 x좌표, 죄수번호, 통과한 문 좌표] 정보 큐에 push
                else: # 이동할 공간이 비었다면
                    q.put([door_num, move_num+1, add_y+y, add_x+x, idx, doors])
                    # [통과한 문 개수, 이동 횟수+1, 이동할 y좌표, 이동할 x좌표, 죄수번호, 통과한 문 좌표] 정보 큐에 push
    for a in opened[0]:
        for b in opened[1]:
            answer = min(answer, len(a.union(b)))
    # 0번 죄수와 1번 죄수의 경우의 수를 모두 고려해서 열어야하는 최소 문의 개수를 구한다.
    return answer # 최소 문의 개수 반환

print("Input")
answer = [] # 답
T = int(input()) # 테스트 케이스 개수
for _ in range(T):
    h, w = tuple(int(x) for x in input().split()) # 감옥 지도 높이, 너비
    prison = [] # 감옥 지도
    prisoner = [] # 죄수 위치
    for y in range(h):
        row = list(input())
        for x in range(w):
            if row[x] == '$': # 죄수
                row[x] = '.'
                prisoner.append((y, x)) # 죄수 위치 저장
        prison.append(row)
    answer.append(simulation(h, w, prison, prisoner)) # 시뮬레이션 후 답 저장
print("\nOutput")
for a in answer:
    print(a) # 답 출력

Input
3
5 9
****#****
*..#.#..*
****.****
*$#.#.#$*
*********
5 11
*#*********
*$*...*...*
*$*.*.*.*.*
*...*...*.*
*********.*
9 9
*#**#**#*
*#**#**#*
*#**#**#*
*#**.**#*
*#*#.#*#*
*$##*##$*
*#*****#*
*.#.#.#.*
*********

Output
4
0
9
